In [3]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

import utils.data_processing_bronze_table
import utils.data_processing_silver_table
import utils.data_processing_gold_table

In [5]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/27 19:24:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/27 19:24:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
# Bronze / Silver / Gold row counts & schemas
for p in [
    "datamart/bronze/lms/",
    "datamart/silver/loan_daily/",
    "datamart/gold/label_store/",
]:
    print("\n===", p, "===")
    df = spark.read.parquet(p)
    print("rows:", df.count())
    df.printSchema()
    df.show(5, truncate=False)



=== datamart/bronze/lms/ ===


25/09/27 19:25:07 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:387)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:443)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1(ParquetFileFormat.scala:493)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1$adapted(ParquetFileFormat.scala:485)
	at org.apache.spark.sql.execution.datasources.SchemaMergeUtils$.$anonfun$mergeSchemasInParallel$2(SchemaMergeUtils.scala:80)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:858)
	at org.apache.spark.

Py4JJavaError: An error occurred while calling o26.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (88c6508da389 executor driver): org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:387)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:443)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1(ParquetFileFormat.scala:493)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1$adapted(ParquetFileFormat.scala:485)
	at org.apache.spark.sql.execution.datas

In [7]:
import os
print([f for f in os.listdir("datamart/bronze/lms/")][:10])


['bronze_loan_daily_2023_01_01.csv', 'bronze_loan_daily_2023_02_01.csv', 'bronze_loan_daily_2023_03_01.csv', 'bronze_loan_daily_2023_04_01.csv', 'bronze_loan_daily_2023_05_01.csv', 'bronze_loan_daily_2023_06_01.csv', 'bronze_loan_daily_2023_07_01.csv', 'bronze_loan_daily_2023_08_01.csv', 'bronze_loan_daily_2023_09_01.csv', 'bronze_loan_daily_2023_10_01.csv']


In [8]:
from pyspark.sql import functions as F

# Bronze (CSV)
bronze = (spark.read
          .option("header", "true")
          .option("inferSchema", "true")
          .csv("datamart/bronze/lms/"))
print("bronze rows:", bronze.count()); bronze.printSchema()

# Silver (Parquet)
silver = spark.read.parquet("datamart/silver/loan_daily/")
print("silver rows:", silver.count()); silver.printSchema()

# Gold (Parquet)
gold = spark.read.parquet("datamart/gold/label_store/")
print("gold rows:", gold.count()); gold.printSchema()


bronze rows: 104288
root
 |-- loan_id: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- loan_start_date: date (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- installment_num: integer (nullable = true)
 |-- loan_amt: integer (nullable = true)
 |-- due_amt: double (nullable = true)
 |-- paid_amt: double (nullable = true)
 |-- overdue_amt: double (nullable = true)
 |-- balance: double (nullable = true)
 |-- snapshot_date: date (nullable = true)



AnalysisException: [UNABLE_TO_INFER_SCHEMA] Unable to infer schema for Parquet. It must be specified manually.

In [9]:
import os, glob

print("Silver dir exists? ", os.path.exists("datamart/silver/loan_daily/"))
print("Silver contents: ", os.listdir("datamart/silver/loan_daily/")[:10])

print("Gold dir exists? ", os.path.exists("datamart/gold/label_store/"))
print("Gold contents: ", os.listdir("datamart/gold/label_store/")[:10])


Silver dir exists?  True
Silver contents:  ['silver_loan_daily_2023_01_01.parquet', 'silver_loan_daily_2023_02_01.parquet', 'silver_loan_daily_2023_03_01.parquet', 'silver_loan_daily_2023_04_01.parquet', 'silver_loan_daily_2023_05_01.parquet', 'silver_loan_daily_2023_06_01.parquet', 'silver_loan_daily_2023_07_01.parquet', 'silver_loan_daily_2023_08_01.parquet', 'silver_loan_daily_2023_09_01.parquet', 'silver_loan_daily_2023_10_01.parquet']
Gold dir exists?  True
Gold contents:  ['gold_label_store_2023_01_01.parquet', 'gold_label_store_2023_02_01.parquet', 'gold_label_store_2023_03_01.parquet', 'gold_label_store_2023_04_01.parquet', 'gold_label_store_2023_05_01.parquet', 'gold_label_store_2023_06_01.parquet', 'gold_label_store_2023_07_01.parquet', 'gold_label_store_2023_08_01.parquet', 'gold_label_store_2023_09_01.parquet', 'gold_label_store_2023_10_01.parquet']


In [10]:
# SILVER
silver = spark.read.parquet("datamart/silver/loan_daily/*.parquet")
print("silver rows:", silver.count()); silver.printSchema()

# GOLD
gold = spark.read.parquet("datamart/gold/label_store/*.parquet")
print("gold rows:", gold.count()); gold.printSchema()


silver rows: 104288
root
 |-- loan_id: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- loan_start_date: date (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- installment_num: integer (nullable = true)
 |-- loan_amt: float (nullable = true)
 |-- due_amt: float (nullable = true)
 |-- paid_amt: float (nullable = true)
 |-- overdue_amt: float (nullable = true)
 |-- balance: float (nullable = true)
 |-- snapshot_date: date (nullable = true)
 |-- mob: integer (nullable = true)
 |-- installments_missed: integer (nullable = true)
 |-- first_missed_date: date (nullable = true)
 |-- dpd: integer (nullable = true)

gold rows: 8974
root
 |-- loan_id: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- label: integer (nullable = true)
 |-- label_def: string (nullable = true)
 |-- snapshot_date: date (nullable = true)



In [3]:
import pandas as pd
pd.read_csv("datamart/bronze/lms/bronze_loan_daily_2023_01_01.csv")["snapshot_date"].unique()



array(['2023-01-01'], dtype=object)

In [4]:
pd.read_csv("datamart/bronze/features/clickstream/bronze_clickstream_2023_01_01.csv").head()

,fe_1,fe_2,fe_3,fe_4,fe_5,fe_6,fe_7,fe_8,fe_9,fe_10,...,fe_16,fe_17,fe_18,fe_19,fe_20,Customer_ID,snapshot_date,ingest_dt,snap_ym,source_file
0,63,118,80,121,55,193,111,112,-101,83,...,114,35,85,-73,76,CUS_0x1037,2023-01-01,2023-01-01,2023-01,feature_clickstream.csv
1,-108,182,123,4,-56,27,25,-6,284,222,...,35,130,94,111,75,CUS_0x1069,2023-01-01,2023-01-01,2023-01,feature_clickstream.csv
2,-13,8,87,166,214,-98,215,152,129,139,...,125,-130,354,17,302,CUS_0x114a,2023-01-01,2023-01-01,2023-01,feature_clickstream.csv
3,-85,45,200,89,128,54,76,51,61,139,...,163,37,207,180,118,CUS_0x1184,2023-01-01,2023-01-01,2023-01,feature_clickstream.csv
4,55,120,226,-86,253,97,107,68,103,126,...,159,-26,104,118,184,CUS_0x1297,2023-01-01,2023-01-01,2023-01,feature_clickstream.csv


In [7]:
import glob
len(glob.glob("datamart/bronze/features/clickstream/bronze_clickstream_*.csv"))

24

In [8]:
len(glob.glob("datamart/bronze/lms/bronze_loan_daily_*.csv"))

24

In [1]:
# quick_verify.py
import glob, pandas as pd

def count_files(pattern): return len(glob.glob(pattern))

print("clickstream files:", count_files("datamart/bronze/features/clickstream/bronze_clickstream_*.csv"))
print("attributes files :", count_files("datamart/bronze/features/attributes/bronze_attributes_*.csv"))
print("financials files :", count_files("datamart/bronze/features/financials/bronze_financials_*.csv"))
print("lms files        :", count_files("datamart/bronze/lms/bronze_loan_daily_*.csv"))

# spot-check one month
df = pd.read_csv("datamart/bronze/lms/bronze_loan_daily_2023_06_01.csv")
print("2023-06-01 snapshot unique:", df["snapshot_date"].unique())


clickstream files: 24
attributes files : 24
financials files : 24
lms files        : 24
2023-06-01 snapshot unique: ['2023-06-01']


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ReadSilver").getOrCreate()

# Example: read attributes Silver for Jan 2023
df = spark.read.parquet("datamart/silver/features/attributes/silver_attributes_2023_01_01.parquet")

df.printSchema()
df.show(5, truncate=False)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/03 17:25:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- customer_id: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- snapshot_date: date (nullable = true)
 |-- ingest_dt: date (nullable = true)
 |-- snap_ym: timestamp (nullable = true)
 |-- source_file: string (nullable = true)

+-----------+----------+-------------+----------+-------------------+-----------------------+
|customer_id|occupation|snapshot_date|ingest_dt |snap_ym            |source_file            |
+-----------+----------+-------------+----------+-------------------+-----------------------+
|CUS_0x1037 |ACCOUNTANT|2023-01-01   |2025-10-03|2023-01-01 00:00:00|features_attributes.csv|
|CUS_0x1069 |ACCOUNTANT|2023-01-01   |2025-10-03|2023-01-01 00:00:00|features_attributes.csv|
|CUS_0x114a |DEVELOPER |2023-01-01   |2025-10-03|2023-01-01 00:00:00|features_attributes.csv|
|CUS_0x1184 |LAWYER    |2023-01-01   |2025-10-03|2023-01-01 00:00:00|features_attributes.csv|
|CUS_0x1297 |MANAGER   |2023-01-01   |2025-10-03|2023-01-01 00:00:00|features_attrib

25/10/03 17:25:47 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
df.select("occupation").distinct().show(100, truncate=False)


+-------------+
|occupation   |
+-------------+
|ACCOUNTANT   |
|DEVELOPER    |
|MECHANIC     |
|ARCHITECT    |
|MEDIA_MANAGER|
|ENGINEER     |
|TEACHER      |
|MUSICIAN     |
|LAWYER       |
|SCIENTIST    |
|WRITER       |
|JOURNALIST   |
|MANAGER      |
|DOCTOR       |
|ENTREPRENEUR |
|NULL         |
+-------------+



In [4]:
spark.stop()

In [1]:
# sanity_financials.py
# Quick checks for Silver financials across a date range.

import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

SILVER_FIN_DIR = "datamart/silver/features/financials"   # no trailing slash pls


def month_range(start_str: str, end_str: str):
    start = datetime.strptime(start_str, "%Y-%m-%d")
    end   = datetime.strptime(end_str,   "%Y-%m-%d")
    cur = datetime(start.year, start.month, 1)
    out = []
    while cur <= end:
        out.append(cur.strftime("%Y-%m-%d"))
        cur += relativedelta(months=1)
    return out


def load_month_df(spark, silver_dir, ds):
    token = datetime.strptime(ds, "%Y-%m-%d").strftime("%Y_%m_%d")
    path = os.path.join(silver_dir, f"silver_financials_{token}.parquet")
    return spark.read.parquet(path).withColumn("ds_token", F.lit(ds))


def dq_for_range(spark, silver_dir, start="2023-01-01", end="2024-12-01"):
    months = month_range(start, end)

    # ---------- Load & union ----------
    dfs = []
    for ds in months:
        try:
            dfs.append(load_month_df(spark, silver_dir, ds))
        except Exception as e:
            print(f"[WARN] Missing or unreadable: {ds} ({e})")
    if not dfs:
        print("[ERROR] No Silver financials found.")
        return
    df = dfs[0]
    for d in dfs[1:]:
        df = df.unionByName(d, allowMissingColumns=True)

    print(f"\n[INFO] Loaded months={len(dfs)} rows={df.count()}")

    # ---------- Basic shape & duplicates ----------
    print("\n[COUNT] rows per ds")
    df.groupBy("ds_token").count().orderBy("ds_token").show(50, False)

    print("\n[DUP] duplicate (customer_id, snapshot_date) rows (should be 0):")
    dup = (
        df.groupBy("customer_id", "snapshot_date", "ds_token")
          .count()
          .filter(F.col("count") > 1)
    )
    print(f"duplicate groups: {dup.count()}")

    # ---------- Type compliance & null rates ----------
    expected_cols = [
        "customer_id","snapshot_date","ingest_dt","snap_ym","source_file",
        "annual_income","monthly_inhand_salary","interest_rate",
        "num_of_loan","delay_from_due_date","num_of_delayed_payment",
        "changed_credit_limit","num_credit_inquiries","credit_mix",
        "outstanding_debt","credit_utilization_ratio","credit_history_age",
        "payment_of_min_amount","total_emi_per_month","amount_invested_monthly",
        "payment_behaviour","monthly_balance","type_of_loan"
    ]
    missing = [c for c in expected_cols if c not in df.columns]
    if missing:
        print("[SCHEMA] Missing columns:", missing)

    print("\n[NULL%] key columns (by month)")
    key_cols = ["annual_income","monthly_inhand_salary","outstanding_debt",
                "interest_rate","credit_utilization_ratio","num_of_loan",
                "num_of_delayed_payment","credit_history_age","credit_mix",
                "payment_behaviour","payment_of_min_amount"]
    exprs = [
        (F.avg(F.when(F.col(c).isNull(), 1.0).otherwise(0.0))*100.0).alias(c)
        for c in key_cols if c in df.columns
    ]
    df.groupBy("ds_token").agg(*exprs).orderBy("ds_token").show(50, False)

    # ---------- Numeric ranges & “2dp” checks ----------
    def show_min_max(name):
        if name in df.columns:
            df.select(
                F.min(F.col(name)).alias("min"),
                F.max(F.col(name)).alias("max"),
                F.avg(F.col(name)).alias("mean")
            ).show(truncate=False)

    print("\n[RANGE] interest_rate % (expect ~0..100)")
    show_min_max("interest_rate")
    out_ir = df.filter((F.col("interest_rate") < 0) | (F.col("interest_rate") > 100))
    print(f"interest_rate out-of-range rows: {out_ir.count()}")

    print("\n[RANGE] credit_history_age (months) (expect non-negative & plausible)")
    show_min_max("credit_history_age")
    out_hist = df.filter((F.col("credit_history_age") < 0) | (F.col("credit_history_age") > 1200))
    print(f"credit_history_age implausible rows (>100y or negative): {out_hist.count()}")

    # “2dp” conformance (allow tiny FP error)
    def two_dp_nonconforming(colname):
        return F.avg(
            F.when(F.abs(F.col(colname) - F.round(F.col(colname), 2)) > F.lit(1e-9), 1.0).otherwise(0.0)
        ).alias(colname)

    dp_cols = ["annual_income","monthly_inhand_salary","outstanding_debt",
               "amount_invested_monthly","monthly_balance"]
    print("\n[2DP%] fraction not rounded to 2 d.p. (should be ~0)")
    df.select([two_dp_nonconforming(c) for c in dp_cols if c in df.columns]).show(truncate=False)

    # ---------- Category hygiene ----------
    if "credit_mix" in df.columns:
        print("\n[CREDIT_MIX] distinct values (expect Good/Standard/Bad or NULL)")
        df.groupBy("credit_mix").count().orderBy(F.desc("count")).show(truncate=False)

    if "payment_behaviour" in df.columns:
        print("\n[PAYMENT_BEHAVIOUR] invalid token '!@9#%8' (should be 0)")
        bad_pb = df.filter(F.col("payment_behaviour") == "!@9#%8")
        print(f"rows with bad token: {bad_pb.count()}")

    # ---------- Cross-field consistency ----------
    # Recompute num_of_loan from type_of_loan to verify
    if "type_of_loan" in df.columns and "num_of_loan" in df.columns:
        tol_norm = F.upper(F.col("type_of_loan").cast("string"))
        tol_norm = F.regexp_replace(tol_norm, r"\bAND\b", ",")
        tol_norm = F.regexp_replace(tol_norm, r"NOT SPECIFIED", "")
        tol_norm = F.regexp_replace(tol_norm, r"\s+", " ")
        tol_norm = F.regexp_replace(tol_norm, r",\s*,", ",")
        tol_norm = F.regexp_replace(tol_norm, r"(^\s*,)|(,\s*$)", "")
        df_check = df.withColumn(
            "_num_from_tol",
            F.when(F.length(F.trim(tol_norm)) == 0, F.lit(0))
             .otherwise(F.size(F.split(tol_norm, r"\s*,\s*"))).cast("int")
        )
        mism = df_check.filter(F.col("_num_from_tol") != F.col("num_of_loan"))
        print("\n[CONSISTENCY] num_of_loan vs parsed type_of_loan mismatches:")
        print(f"mismatch rows: {mism.count()}")
        mism.select("ds_token","customer_id","type_of_loan","num_of_loan","_num_from_tol").show(10, False)

    # ---------- Value reasonableness ----------
    nonneg_cols = ["annual_income","monthly_inhand_salary","outstanding_debt",
                   "total_emi_per_month","amount_invested_monthly","monthly_balance",
                   "num_of_loan","num_of_delayed_payment","delay_from_due_date",
                   "num_credit_inquiries","credit_history_age","credit_utilization_ratio"]
    print("\n[NEG%] numeric columns with negatives (should be ~0 except credit_change can be ±)")
    exprs2 = [
        (F.avg(F.when(F.col(c) < 0, 1.0).otherwise(0.0))*100.0).alias(c)
        for c in nonneg_cols if c in df.columns
    ]
    if exprs2:
        df.select(*exprs2).show(truncate=False)

    # ---------- Spot check examples ----------
    print("\n[SAMPLE] interest_rate out-of-range examples")
    out_ir.select("ds_token","customer_id","interest_rate").show(10, False)

    print("\n[SAMPLE] 2dp non-conforming amounts")
    expr_flag = (F.abs(F.col("annual_income") - F.round(F.col("annual_income"), 2)) > F.lit(1e-9))
    if "annual_income" in df.columns:
        df.filter(expr_flag).select("ds_token","customer_id","annual_income").show(10, False)

    print("\n[OK] Sanity checks complete.")


if __name__ == "__main__":
    spark = (
        SparkSession.builder.appName("DQ-Silver-Financials")
        .master("local[*]")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("ERROR")

    dq_for_range(
        spark,
        silver_dir=SILVER_FIN_DIR,
        start="2023-01-01",
        end="2024-12-01"
    )

    spark.stop()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/03 19:39:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                


[INFO] Loaded months=24 rows=11974

[COUNT] rows per ds


+----------+-----+
|ds_token  |count|
+----------+-----+
|2023-01-01|530  |
|2023-02-01|501  |
|2023-03-01|506  |
|2023-04-01|510  |
|2023-05-01|521  |
|2023-06-01|517  |
|2023-07-01|471  |
|2023-08-01|481  |
|2023-09-01|454  |
|2023-10-01|487  |
|2023-11-01|491  |
|2023-12-01|489  |
|2024-01-01|485  |
|2024-02-01|518  |
|2024-03-01|511  |
|2024-04-01|513  |
|2024-05-01|491  |
|2024-06-01|498  |
|2024-07-01|505  |
|2024-08-01|543  |
|2024-09-01|493  |
|2024-10-01|456  |
|2024-11-01|488  |
|2024-12-01|515  |
+----------+-----+


[DUP] duplicate (customer_id, snapshot_date) rows (should be 0):


duplicate groups: 0

[NULL%] key columns (by month)


+----------+-------------+---------------------+----------------+-------------+------------------------+-----------+----------------------+------------------+------------------+------------------+---------------------+
|ds_token  |annual_income|monthly_inhand_salary|outstanding_debt|interest_rate|credit_utilization_ratio|num_of_loan|num_of_delayed_payment|credit_history_age|credit_mix        |payment_behaviour |payment_of_min_amount|
+----------+-------------+---------------------+----------------+-------------+------------------------+-----------+----------------------+------------------+------------------+------------------+---------------------+
|2023-01-01|0.0          |0.0                  |0.0             |0.0          |0.0                     |0.0        |0.0                   |0.0               |20.754716981132077|7.169811320754717 |0.0                  |
|2023-02-01|0.0          |0.0                  |0.0             |0.0          |0.0                     |0.0        |0.0     

...

+----------+-----+
|credit_mix|count|
+----------+-----+
|Standard  |4287 |
|Good      |2918 |
|NULL      |2505 |
|Bad       |2264 |
+----------+-----+


[PAYMENT_BEHAVIOUR] invalid token '!@9#%8' (should be 0)


rows with bad token: 0

[CONSISTENCY] num_of_loan vs parsed type_of_loan mismatches:


mismatch rows: 2314
+----------+-----------+-----------------------------------------------------------------------------------------------------+-----------+-------------+
|ds_token  |customer_id|type_of_loan                                                                                         |num_of_loan|_num_from_tol|
+----------+-----------+-----------------------------------------------------------------------------------------------------+-----------+-------------+
|2023-01-01|CUS_0x1325 |Not Specified, Personal Loan, Auto Loan, Not Specified, and Debt Consolidation Loan                  |3          |4            |
|2023-01-01|CUS_0x13a8 |NULL                                                                                                 |0          |-1           |
|2023-01-01|CUS_0x1630 |Not Specified, Student Loan, Personal Loan, Not Specified, Not Specified, and Debt Consolidation Loan|3          |4            |
|2023-01-01|CUS_0x169c |NULL                                  

+-------------+---------------------+----------------+-------------------+-----------------------+---------------+-----------+----------------------+-------------------+--------------------+------------------+------------------------+
|annual_income|monthly_inhand_salary|outstanding_debt|total_emi_per_month|amount_invested_monthly|monthly_balance|num_of_loan|num_of_delayed_payment|delay_from_due_date|num_credit_inquiries|credit_history_age|credit_utilization_ratio|
+-------------+---------------------+----------------+-------------------+-----------------------+---------------+-----------+----------------------+-------------------+--------------------+------------------+------------------------+
|0.0          |0.0                  |0.0             |0.0                |0.0                    |0.0            |0.0        |0.0                   |0.7098713880073493 |0.0                 |0.0               |0.0                     |
+-------------+---------------------+----------------+------

+--------+-----------+-------------+
|ds_token|customer_id|annual_income|
+--------+-----------+-------------+
+--------+-----------+-------------+


[OK] Sanity checks complete.


In [2]:
# sanity_financials.py
# Quick checks for Silver financials across a date range (aligned to final Silver rules).
#
# - Loads each month's silver_financials_<YYYY_MM_DD>.parquet and unions them
# - Counts rows/month, duplicate (customer_id, snapshot_date), null rates
# - Range checks: interest_rate in 0..100 (%), credit_history_age months plausible
# - 2 d.p. conformance for money fields
# - Category hygiene for credit_mix and payment_behaviour
# - Consistency: num_of_loan == (raw comma count + 1), with NULL/empty -> 0
# - Reasonableness: negatives for non-negative numeric fields

import os
from datetime import datetime
from dateutil.relativedelta import relativedelta

import pyspark.sql.functions as F
from pyspark.sql import SparkSession

SILVER_FIN_DIR = "datamart/silver/features/financials"   # no trailing slash


def month_range(start_str: str, end_str: str):
    start = datetime.strptime(start_str, "%Y-%m-%d")
    end   = datetime.strptime(end_str,   "%Y-%m-%d")
    cur = datetime(start.year, start.month, 1)
    out = []
    while cur <= end:
        out.append(cur.strftime("%Y-%m-%d"))
        cur += relativedelta(months=1)
    return out


def load_month_df(spark, silver_dir, ds):
    token = datetime.strptime(ds, "%Y-%m-%d").strftime("%Y_%m_%d")
    path = os.path.join(silver_dir, f"silver_financials_{token}.parquet")
    return spark.read.parquet(path).withColumn("ds_token", F.lit(ds))


def dq_for_range(spark, silver_dir, start="2023-01-01", end="2024-12-01"):
    months = month_range(start, end)

    # ---------- Load & union ----------
    dfs = []
    for ds in months:
        try:
            dfs.append(load_month_df(spark, silver_dir, ds))
        except Exception as e:
            print(f"[WARN] Missing or unreadable: {ds} ({e})")
    if not dfs:
        print("[ERROR] No Silver financials found.")
        return

    df = dfs[0]
    for d in dfs[1:]:
        df = df.unionByName(d, allowMissingColumns=True)

    total_rows = df.count()
    print(f"\n[INFO] Loaded months={len(dfs)} rows={total_rows}")

    # ---------- Basic shape & duplicates ----------
    print("\n[COUNT] rows per ds")
    df.groupBy("ds_token").count().orderBy("ds_token").show(50, False)

    print("\n[DUP] duplicate (customer_id, snapshot_date) groups (should be 0):")
    dup = (
        df.groupBy("customer_id", "snapshot_date", "ds_token")
          .count()
          .filter(F.col("count") > 1)
    )
    print(f"duplicate groups: {dup.count()}")

    # ---------- Schema presence ----------
    expected_cols = [
        "customer_id","snapshot_date","ingest_dt","snap_ym","source_file",
        "annual_income","monthly_inhand_salary","interest_rate",
        "num_of_loan","delay_from_due_date","num_of_delayed_payment",
        "changed_credit_limit","num_credit_inquiries","credit_mix",
        "outstanding_debt","credit_utilization_ratio","credit_history_age",
        "payment_of_min_amount","total_emi_per_month","amount_invested_monthly",
        "payment_behaviour","monthly_balance","type_of_loan"
    ]
    missing = [c for c in expected_cols if c not in df.columns]
    if missing:
        print("[SCHEMA] Missing columns:", missing)

    # ---------- Null rates (by month) ----------
    print("\n[NULL%] key columns (by month)")
    key_cols = ["annual_income","monthly_inhand_salary","outstanding_debt",
                "interest_rate","credit_utilization_ratio","num_of_loan",
                "num_of_delayed_payment","credit_history_age","credit_mix",
                "payment_behaviour","payment_of_min_amount"]
    exprs = [
        (F.avg(F.when(F.col(c).isNull(), 1.0).otherwise(0.0))*100.0).alias(c)
        for c in key_cols if c in df.columns
    ]
    if exprs:
        df.groupBy("ds_token").agg(*exprs).orderBy("ds_token").show(50, False)

    # ---------- Ranges ----------
    def show_min_max_mean(name, title=None):
        if name in df.columns:
            print(f"\n[RANGE] {title or name}")
            df.select(
                F.min(F.col(name)).alias("min"),
                F.max(F.col(name)).alias("max"),
                F.avg(F.col(name)).alias("mean")
            ).show(truncate=False)

    # interest_rate in 0..100
    show_min_max_mean("interest_rate", "interest_rate % (expect 0..100)")
    out_ir = df.filter((F.col("interest_rate") < 0) | (F.col("interest_rate") > 100))
    print(f"interest_rate out-of-range rows: {out_ir.count()}")

    # credit_history_age months plausible (>=0, <= 1200 ~ 100y)
    show_min_max_mean("credit_history_age", "credit_history_age months (expect >=0 & <=1200)")
    out_hist = df.filter((F.col("credit_history_age") < 0) | (F.col("credit_history_age") > 1200))
    print(f"credit_history_age implausible rows (>100y or negative): {out_hist.count()}")

    # ---------- “2dp” conformance for money fields ----------
    def two_dp_nonconforming(colname):
        return F.avg(
            F.when(F.abs(F.col(colname) - F.round(F.col(colname), 2)) > F.lit(1e-9), 1.0).otherwise(0.0)
        ).alias(colname)

    dp_cols = ["annual_income","monthly_inhand_salary","outstanding_debt",
               "amount_invested_monthly","monthly_balance"]
    print("\n[2DP%] fraction not rounded to 2 d.p. (should be ~0)")
    df.select([two_dp_nonconforming(c) for c in dp_cols if c in df.columns]).show(truncate=False)

    # ---------- Category hygiene ----------
    if "credit_mix" in df.columns:
        print("\n[CREDIT_MIX] distinct values (expect Good/Standard/Bad or NULL)")
        df.groupBy("credit_mix").count().orderBy(F.desc("count")).show(truncate=False)

    if "payment_behaviour" in df.columns:
        print("\n[PAYMENT_BEHAVIOUR] invalid token '!@9#%8' (should be 0)")
        bad_pb = df.filter(F.col("payment_behaviour") == "!@9#%8")
        print(f"rows with bad token: {bad_pb.count()}")

    # ---------- Consistency: num_of_loan vs (commas + 1) on RAW type_of_loan ----------
    if "type_of_loan" in df.columns and "num_of_loan" in df.columns:
        tol_raw = F.col("type_of_loan").cast("string")
        is_zero = (
            tol_raw.isNull() |
            (F.length(tol_raw) == 0) |
            (F.upper(F.trim(tol_raw)) == F.lit("NULL"))
        )
        comma_count = F.length(tol_raw) - F.length(F.regexp_replace(tol_raw, ",", ""))
        df_check = df.withColumn(
            "_num_from_tol",
            F.when(is_zero, F.lit(0)).otherwise((comma_count + F.lit(1)).cast("int"))
        )
        mism = df_check.filter(F.col("_num_from_tol") != F.col("num_of_loan"))
        print("\n[CONSISTENCY] num_of_loan vs (commas+1) mismatches:")
        print(f"mismatch rows: {mism.count()}")
        mism.select("ds_token","customer_id","type_of_loan","num_of_loan","_num_from_tol").show(10, False)

    # ---------- Reasonableness: non-negative numeric fields ----------
    nonneg_cols = ["annual_income","monthly_inhand_salary","outstanding_debt",
                   "total_emi_per_month","amount_invested_monthly","monthly_balance",
                   "num_of_loan","num_of_delayed_payment","num_credit_inquiries",
                   "credit_history_age","credit_utilization_ratio"]
    print("\n[NEG%] numeric columns with negatives (should be ~0; delay_from_due_date may be ±)")
    exprs2 = [
        (F.avg(F.when(F.col(c) < 0, 1.0).otherwise(0.0))*100.0).alias(c)
        for c in nonneg_cols if c in df.columns
    ]
    if exprs2:
        df.select(*exprs2).show(truncate=False)

    # delay_from_due_date separate (can be negative for early payments)
    if "delay_from_due_date" in df.columns:
        print("\n[DELAY_FROM_DUE_DATE] share negative (early payments)")
        df.select((F.avg(F.when(F.col("delay_from_due_date") < 0, 1.0).otherwise(0.0))*100.0)
                  .alias("neg_%")).show(truncate=False)

    # ---------- Spot samples ----------
    print("\n[SAMPLE] interest_rate out-of-range examples")
    out_ir.select("ds_token","customer_id","interest_rate").show(10, False)

    print("\n[OK] Sanity checks complete.")


if __name__ == "__main__":
    spark = (
        SparkSession.builder.appName("DQ-Silver-Financials")
        .master("local[*]")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("ERROR")

    dq_for_range(
        spark,
        silver_dir=SILVER_FIN_DIR,
        start="2023-01-01",
        end="2024-12-01"
    )

    spark.stop()



[INFO] Loaded months=24 rows=11974

[COUNT] rows per ds


+----------+-----+
|ds_token  |count|
+----------+-----+
|2023-01-01|530  |
|2023-02-01|501  |
|2023-03-01|506  |
|2023-04-01|510  |
|2023-05-01|521  |
|2023-06-01|517  |
|2023-07-01|471  |
|2023-08-01|481  |
|2023-09-01|454  |
|2023-10-01|487  |
|2023-11-01|491  |
|2023-12-01|489  |
|2024-01-01|485  |
|2024-02-01|518  |
|2024-03-01|511  |
|2024-04-01|513  |
|2024-05-01|491  |
|2024-06-01|498  |
|2024-07-01|505  |
|2024-08-01|543  |
|2024-09-01|493  |
|2024-10-01|456  |
|2024-11-01|488  |
|2024-12-01|515  |
+----------+-----+


[DUP] duplicate (customer_id, snapshot_date) groups (should be 0):


duplicate groups: 0

[NULL%] key columns (by month)


+----------+-------------+---------------------+----------------+-------------+------------------------+-----------+----------------------+------------------+------------------+------------------+---------------------+
|ds_token  |annual_income|monthly_inhand_salary|outstanding_debt|interest_rate|credit_utilization_ratio|num_of_loan|num_of_delayed_payment|credit_history_age|credit_mix        |payment_behaviour |payment_of_min_amount|
+----------+-------------+---------------------+----------------+-------------+------------------------+-----------+----------------------+------------------+------------------+------------------+---------------------+
|2023-01-01|0.0          |0.0                  |0.0             |0.0          |0.0                     |0.0        |0.0                   |0.0               |20.754716981132077|7.169811320754717 |0.0                  |
|2023-02-01|0.0          |0.0                  |0.0             |0.0          |0.0                     |0.0        |0.0     

+----+-----+-----------------+
|min |max  |mean             |
+----+-----+-----------------+
|1.01|100.0|17.48090863537665|
+----+-----+-----------------+

interest_rate out-of-range rows: 0

[RANGE] credit_history_age months (expect >=0 & <=1200)


...

rows with bad token: 0

[CONSISTENCY] num_of_loan vs (commas+1) mismatches:


mismatch rows: 0


+--------+-----------+------------+-----------+-------------+
|ds_token|customer_id|type_of_loan|num_of_loan|_num_from_tol|
+--------+-----------+------------+-----------+-------------+
+--------+-----------+------------+-----------+-------------+


[NEG%] numeric columns with negatives (should be ~0; delay_from_due_date may be ±)


+-------------+---------------------+----------------+-------------------+-----------------------+---------------+-----------+----------------------+--------------------+------------------+------------------------+
|annual_income|monthly_inhand_salary|outstanding_debt|total_emi_per_month|amount_invested_monthly|monthly_balance|num_of_loan|num_of_delayed_payment|num_credit_inquiries|credit_history_age|credit_utilization_ratio|
+-------------+---------------------+----------------+-------------------+-----------------------+---------------+-----------+----------------------+--------------------+------------------+------------------------+
|0.0          |0.0                  |0.0             |0.0                |0.0                    |0.0            |0.0        |0.0                   |0.0                 |0.0               |0.0                     |
+-------------+---------------------+----------------+-------------------+-----------------------+---------------+-----------+--------------

+------------------+
|neg_%             |
+------------------+
|0.7098713880073493|
+------------------+


[SAMPLE] interest_rate out-of-range examples
+--------+-----------+-------------+
|ds_token|customer_id|interest_rate|
+--------+-----------+-------------+
+--------+-----------+-------------+


[OK] Sanity checks complete.
